In [2]:
# Kipris키프리스에서 인용/비인용, 패밀리 정보 수집하기 / 엑셀로 저장하기

#Step 1. 필요한 모듈을 로딩합니다
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time 
         
#from webdriver_manager.chrome import ChromeDriverManager

#Step 2. 사용자에게 검색 관련 정보들을 입력 받습니다.
#키프리스 로그인이 필요할 시 입력.
#login_ID = "202214289" #input(' ID를 입력하세요. : ')
#login_password = "5L0XNBD5#" #input(' 비밀번호를 입력하세요. : ')


#Step 3. 수집된 데이터를 저장할 파일 이름 입력받기 
#fx_name = input('데이터를 저장할 xls형식의 파일명을 입력하세요. (예:\\Users\\dk\\KONKUK\\pypyreal\\everytime\\values\\references.xlsx): ')

free_query = input('자유검색(전문) ex) 헤어드라이기 :    ') # 헤어드라이기
applicant = input('출원인 ex) 다이슨 :    ')# 다이슨

#Step 4. 크롬 드라이버 설정 및 웹 페이지 열기
s = Service("/Users/dk/KONKUK/pypyreal/everytime/chromedriver")
driver = webdriver.Chrome(service=s)


#절대경로 지정
#driver = webdriver.Chrome(executable_path=r"C:\\Users\\dk\\PYTHON\\pypyreal\\chromedriver")

url = 'http://www.kipris.or.kr/khome/main.jsp'
driver.get(url)
time.sleep(1)
driver.maximize_window()

#Step 5. 입력받은 아이디 비번 입력 후 로그인
#element = driver.find_element(By.ID,'edId')
#driver.find_element(By.ID,'edId').click()
#element.send_keys(login_ID)

#element = driver.find_element(By.ID,'edPass')    #element 관련 문법 찾기
#driver.find_element(By.ID,'edPass').click()       #   5L0XNBD5#
#element.send_keys(login_password)
#element.send_keys("\n")
##element.send_keys("ENTER") 된다고 해서 써봤는데 안됨. Keys 도 import 했는뎁..      #원래 주석처리 되어있던건 ##두번씀. 로그인 필요하면 #만 지우고 ##는 #로.
#time.sleep(5)


#Step 6. 특허.실용신안 -> 스마트 검색 "이곳을 클릭해주세요" -> 행정상태 : "전체" 클릭해서 모두 취소, 이후 "등록"만 클릭 -> 자유검색 : "input = 헤어드라이기 ", scroll down 필요할 수도? 출원인 : "input = 다이슨" -> 아래로 내려서, 검색하기 버튼 클릭
#element = driver.find_element(By.ID,'treeview1_label_24') #키값 보낼때만 element 정의 필요
# XPath로 요소 찾기 및 클릭
# 여기서는 `a` 태그의 `href` 속성이 특정 URL과 일치하는 요소를 찾고 클릭
xpath = "//a[@href='http://kpat.kipris.or.kr/kpat/searchLogina.do?next=MainSearch']"
driver.find_element(By.XPATH, xpath).click()
time.sleep(1)

element = driver.find_element(By.ID, "ToggleSmartFinder").click()
time.sleep(1)
element = driver.find_element(By.ID, "smartHangjung00").click()
time.sleep(1)
element = driver.find_element(By.ID, "smartHangjung07").click()
time.sleep(1)

#처음에 받아온 input 값을 여기서 입력
element = driver.find_element(By.ID, "KW")
element.send_keys(free_query)
time.sleep(0.5)
# 버튼 요소를 찾기
query_button = driver.find_element(By.ID, "btnItemizedSearch")
# 버튼이 보이도록 스크롤
driver.execute_script("arguments[0].scrollIntoView();", query_button)
# 그전에 출원인 입력
element = driver.find_element(By.ID, "AP")
element.send_keys(applicant)
time.sleep(0.5)
# 버튼 클릭
query_button.click()
time.sleep(2)
#----------여기서부터 데이터 수집 시작----------

# 모든 링크 요소를 찾기 - `href` 속성에 `openSubDetail`을 포함하는 <a> 태그    
# 링크 요소를 찾기 - `openSubDetail`이 포함된 링크
# h1 구조 내에서 링크 요소들 가져오기
links = driver.find_elements(By.XPATH, "//h1[@class='stitle']/a[contains(@href, 'javascript:openSubDetail')]")

# 링크 개수 확인 및 href 출력
print(f"찾은 링크 개수: {len(links)}")
for link in links:
    print(link.get_attribute('href'))

# 각 링크에 대해 JavaScript로 함수를 직접 호출

while links:
    # 첫 번째 링크를 가져오고 리스트에서 제거
    link = links.pop(0)
    
    # JavaScript로 링크의 href 속성의 함수를 실행
    driver.execute_script(link.get_attribute('href'))
    
    # 새 창으로 전환
    driver.switch_to.window(driver.window_handles[-1])
    time.sleep(1)
    
    # 인용/피인용이 나올 때까지 대기하고 클릭
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#liViewSub07 a"))
    )
    driver.find_element(By.CSS_SELECTOR, "#liViewSub07 a").click()
    
    time.sleep(2)  # 필요한 작업 수행
    
    # 작업 후 새 창 닫기
    driver.close()
    driver.switch_to.window(driver.window_handles[0])  # 원래 창으로 돌아오기
    
    # 다음 링크를 찾기 위해 links를 다시 초기화할 필요 없음
    time.sleep(1)


# #여기에 몇년도 수강편람을 찾는지 묻기
    
# frame1 = driver.find_element(By.ID, 'iframe1')
# driver.switch_to.frame(frame1)

# #WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH,"//iframe[@id="iframe_id and @src='iframe_src'"]")))

# frame3 = driver.find_element(By.ID, 'ifrForm')
# driver.switch_to.frame(frame3)
# time.sleep(2)


# element = driver.find_element(By.ID,'ipF_YY')
# driver.find_element(By.ID,'ipF_YY').click()
# time.sleep(3)
# driver.find_element(By.ID,'ipF_YY').send_keys(Keys.COMMAND + 'a') #글자를 입력할 수 있는 요소의 글자를 지움
# time.sleep(3)
# driver.find_element(By.ID,'ipF_YY').send_keys(Keys.DELETE)
# time.sleep(3)
# element.send_keys('2023')
# time.sleep(3)


# #1. 1학기 정규, 2. 1학기 계절, 3. 2학기 정규, 4. 2학기 계절 중 어느 학기를 찾는지 묻기 아니그냥 처음에 다 묻자
# driver.find_element(By.ID,"sbF_SHTM_CD_button").click()
# driver.find_element(By.ID,"sbF_SHTM_CD_itemTable_0").click()  #1학기 정규

# driver.find_element(By.ID, "sbF_FCLT_CD_button").click()  #학과 선택 시작
# #학과 나열 여기서 index를 가지고 tr > td 에서 반복문 돌릴수 있게

# department = [ ]
# grade = [ ]
# part = [ ]
# code = [ ]
# number = [ ]
# title = [ ]
# credit = [ ]
# hour = [ ]
# libe_part = [ ]
# professor = [ ]
# timee = [ ]
# place = [ ]
# reference= [ ]


# driver.switch_to.parent_frame() #iframe1으로 
# driver.find_element(By.ID,"tgSelect_left").click() #조회
# time.sleep(3)
# #department_list = driver.find_all('td', attrs={"class" : "w2table_sb_col"})
# #print(department_list)
# frame3 = driver.find_element(By.ID, 'ifrForm')
# driver.switch_to.frame(frame3)
# time.sleep(1)

# #temp = driver.find_element(By.ID, 'gdM0_F0_cell_0_1')
# #print(temp.text) IT융합공학과




# #수프 먹자... 이제 텍스트 추출하기

# from bs4 import BeautifulSoup
# html = driver.page_source
# soup = BeautifulSoup(html, 'html.parser')

# tbody = soup.find(id='gdM0_F0_body_tbody')  #tbody > tr > td 하지 않고 바로 tbody > td로. tr이 find_all 할 수 없음. 싱글 엘리먼트가 아님(?)
# tds = tbody.find_all('td')

# for td in tds[:15]:
#     if not td.text:
#         print(None)
#     print(td.text)
    
    

    
    

찾은 링크 개수: 9
javascript:openSubDetail('1020167015947', '0', '', 'biblio', '30', 'View01', '')
javascript:openSubDetail('1020217009637', '1', '', 'biblio', '30', 'View01', '')
javascript:openSubDetail('1020167001255', '2', '', 'biblio', '30', 'View01', '')
javascript:openSubDetail('1020187026902', '3', '', 'biblio', '30', 'View01', '')
javascript:openSubDetail('1020147027214', '4', '', 'biblio', '30', 'View01', '')
javascript:openSubDetail('1020217002569', '5', '', 'biblio', '30', 'View01', '')
javascript:openSubDetail('1020187031802', '6', '', 'biblio', '30', 'View01', '')
javascript:openSubDetail('1020157028215', '7', '', 'biblio', '30', 'View01', '')
javascript:openSubDetail('1020197034590', '8', '', 'biblio', '30', 'View01', '')


KeyboardInterrupt: 

In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

# ChromeDriver 경로 설정
s = Service("/Users/dk/KONKUK/pypyreal/everytime/chromedriver")
driver = webdriver.Chrome(service=s)

# 원하는 URL 열기
driver.get("https://www.google.com")


In [ ]:
temp = driver.find_element(By.ID, 'gdM0_F0_cell_0_1')
for department in temp:
    print(department.text)

In [ ]:

#Step 7.Beautiful Soup 로 본문 내용만 추출하기 학부 선택
from bs4 import BeautifulSoup

html_1 = driver.page_source
soup_1 = BeautifulSoup(html_1, 'html.parser')

department_list = soup_1.find_all('td', attrs={"class" : "w2table_sb_col"})
print(department_list)



In [ ]:

for i in department_list:
    soup.select('#') # 아이디 안에있는 태그를 전부 가져온다음 하나씩 클릭하게
    
    driver.find_element_by_class('w2table_sb_col').click() #????? 인덱스나 col번호로 하나씩 바뀌었으면 좋겠는데
    
    for j in 






#티스토리 북마크 참고 !! 나중에 꼭 볼것 반복문, 데이터프레임 만들어서 저장까지 https://yoo-young.tistory.com/31
exchange_data = []
baseurl = 'https://finance.naver.com/'

for item in exchangeList:
    data = {
        'title':item.select_one(".h_lst").text,
        'exChange': item.select_one('.value').text,
        "change" : item.select_one('.change').text,
        "updown" : item.select_one('.head_info.point_up > .blind').text if item.select_one('.head_info.point_up > .blind') != None else item.select_one('.head_info.point_dn > .blind').text,
        "link": baseurl + item.select_one('a').get('href')
    }
    print(data)
    exchange_data.append(data)










#Step 8. 총 검색 건수를 보여주고 수집할 건수 입력받기
import math
total_cnt = soup_1.find('div','searchBox pd').find('span','num').get_text()
print('검색하신 키워드 %s (으)로 총 %s 건의 학위논문이 검색되었습니다' %(query_txt,total_cnt))
collect_cnt = int(input('이 중에서 몇 건을 수집하시겠습니까?: '))
collect_page_cnt = math.ceil(collect_cnt / 10)
print('%s 건의 데이터를 수집하기 위해 %s 페이지의 게시물을 조회합니다.' %(collect_cnt,collect_page_cnt))
print('=' *80)

#Step 9. 각 항목별로 데이터를 추출하여 리스트에 저장하기
no2 = [ ]        #번호 저장
title2 = [ ]     #논문제목 저장
writer2 = [ ]    #논문저자 저장
org2 = [ ]       #소속기관 저장
no = 1

for a in range(1, collect_page_cnt + 1) :
    
    html_2 = driver.page_source
    soup_2 = BeautifulSoup(html_2, 'html.parser')

    content_2 = soup_2.find('div','srchResultListW').find_all('li')
    
    for b in content_2 :    
        #1. 논문제목 있을 경우만
        try :
            title = b.find('div','cont').find('p','title').get_text()
        except :
            continue
        else :
            f = open(fx_name, 'a' , encoding="UTF-8")
            print('1.번호:',no)
            no2.append(no)
            f.write('\n'+'1.번호:' + str(no))

            print('2.논문제목:',title)
            title2.append(title)
            f.write('\n' + '2.논문제목:' + title)
            
            writer = b.find('span','writer').get_text()
            print('3.저자:',writer)
            writer2.append(writer)
            f.write('\n' + '3.저자:' + writer)

            org = b.find('span','assigned').get_text()
            print('4.소속기관:' , org)
            org2.append(org)
            f.write('\n' + '4.소속기관:' + org + '\n')
            
            f.close( )
            
            no += 1
            print("\n")
            
            if no > collect_cnt :
                break

            time.sleep(1)
    a += 1 
    b = str(a)

    try :
        driver.find_element(By.LINK_TEXT ,'%s' %b).click() 
    except :
        driver.find_element(By.LINK_TEXT('다음 페이지로')).click()
        
print("요청하신 작업이 모두 완료되었습니다")

# Step 10. 수집된 데이터를 xls와 csv 형태로 저장하기
import pandas as pd 

df = pd.DataFrame()
df['번호']=no2
df['제목']=pd.Series(title2)
df['저자']=pd.Series(writer2)
df['소속(발행)기관']=pd.Series(org2)

# xls 형태로 저장하기
df.to_excel(fx_name,index=False, encoding="utf-8" , engine='openpyxl')

# csv 형태로 저장하기
df.to_csv(fc_name,index=False, encoding="utf-8-sig")

print('요청하신 데이터 수집 작업이 정상적으로 완료되었습니다')